In [ ]:
import pyodbc
import pandas as pd
# Set up your connection string
from dotenv import load_dotenv
import os 

load_dotenv()

server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')

# Define the connection string for SQL Server (Azure)
conn_str = (
    f'DRIVER={{ODBC Driver 18 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)

# Establish the connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connected to the Azure SQL Database successfully!")

except pyodbc.Error as e:
    print(f"Error connecting to the database: {e}")


query = "SELECT * FROM SalesLT.customer"

try:
    # Read data into a DataFrame
    df = pd.read_sql(query, conn)
    print("Data retrieved successfully!")
except Exception as e:
    print(f"Error reading data: {e}")

conn.close()
print("Connection closed.")

### Prueba ###
data=df.head(5)
data2=data.copy()


In [ ]:
def prepare_data(df:pd.DataFrame) -> pd.DataFrame:
    df = df[['FirstName', 'ModifiedDate']].sort_values(by='ModifiedDate')
    df['FirstNameLen'] = df['FirstName'].str.len()
    df["ModifiedDate"] = pd.to_datetime(df["ModifiedDate"]) - pd.to_datetime(df["ModifiedDate"]).min()
    df["ModifiedDate"] = df["ModifiedDate"].dt.days

data = prepare_data(data)
data_dict = data.to_dict(orient='list')
    

In [ ]:
import json

#Formateado para la API:
data_json = json.dumps({"data": [data_dict]})

suri = open("uri2.json", "r")
scoring_uri = json.load(suri)["URI"][0]
suri.close()

In [ ]:
headers = {"Content-Type": "application/json"}
import requests
response = requests.post(scoring_uri, data=data_json, headers=headers)

if response.status_code == 200:
  result = json.loads(response.json())
  print('Predicciones del modelo:',result)
  data["Exited"] = result
  print('Variable objetivo real:',list(data2['ModifiedDate']))
  #Print la precisión del modelo usando el porcentaje de aciertos
  aciertos = 0
  for i in range(len(data)):
    if data["Exited"][i] == list(data2['Bankrupt?'])[i]:
      aciertos += 1
  precision = aciertos / len(data) * 100
  print(f"Precisión del modelo: {precision:.2f}%")
else:
  print(f"Error: {response.text}")